In [ ]:
import tensorflow as tf
import numpy as np
from vizdoom import *

import random
import time
from skimage import transform

from collections import deque
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!ls scenarios/

In [ ]:
def create_environment():
    game = DoomGame()
    game.load_config("scenarios/basic.cfg")
    game.set_window_visible(False)
    
    game.init()
    
    left = [1, 0, 0]
    right = [0, 1, 0]
    shoot = [0, 0, 1]
    possible_actions = [left, right, shoot]
    
    return game, possible_actions

In [ ]:
def test_environment(game, actions):
    episodes = 10
    for i in range(episodes):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice(actions)
            # print(action)
            reward = game.make_action(action)
            # print ("\treward:", reward)
            #time.sleep(0.007)
        print ("Result:", game.get_total_reward())
        #time.sleep(1)
    game.close()

In [ ]:
game, possible_actions = create_environment()

In [ ]:
test_environment(game, possible_actions)

In [ ]:
def preprocess_frame(frame):
    cropped_frame = frame[30:-10,30:-30]
    
    normalized_frame = cropped_frame/255.0
    
    resized_frame = transform.resize(normalized_frame, [84,84])
    
    return resized_frame

In [ ]:
stack_size = 4

def init_deque():
    # Initialize deque with zero-images one array for each image
    return deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

stacked_frames = init_deque()

def stack_frames(stacked_frames, state, is_new_episode):
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = init_deque()
        
        # Because we're in a new episode, copy  the same frame 4x.
        for _ in range(4):
            stacked_frames.append(frame)
            
    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)
        
    # Stack the frames.
    stacked_state = np.stack(stacked_frames, axis=2)
    
    return stacked_state, stacked_frames
